In [5]:
import tensorflow as tf
import torch
import pandas as pd

In [11]:
data_path = "./data/mail_data.csv"
df = pd.read_csv(data_path)

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Get the email content
X = df['Message']

# Tokenize the text
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

# Pad the sequences
padded_sequences = pad_sequences(sequences, padding='post')


In [22]:
from sklearn.preprocessing import MultiLabelBinarizer

# Use MultiLabelBinarizer to encode the labels
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Category'])

# Display the transformed labels
print(y)

[[1 0]
 [1 0]
 [0 1]
 ...
 [1 0]
 [1 0]
 [1 0]]


In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

# Define the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=padded_sequences.shape[1]),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(len(mlb.classes_), activation='sigmoid')  # Output for multicategory classification
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, y, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10


2024-09-20 20:33:41.162022: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


140/140 [==============================] - 1s 2ms/step - loss: 0.5544 - accuracy: 0.8600 - val_loss: 0.3891 - val_accuracy: 0.8700
Epoch 2/10
140/140 [==============================] - 0s 1ms/step - loss: 0.3771 - accuracy: 0.8649 - val_loss: 0.3614 - val_accuracy: 0.8700
Epoch 3/10
140/140 [==============================] - 0s 1ms/step - loss: 0.3660 - accuracy: 0.8649 - val_loss: 0.3517 - val_accuracy: 0.8700
Epoch 4/10
140/140 [==============================] - 0s 1ms/step - loss: 0.3520 - accuracy: 0.8649 - val_loss: 0.3346 - val_accuracy: 0.8700
Epoch 5/10
140/140 [==============================] - 0s 1ms/step - loss: 0.3283 - accuracy: 0.8649 - val_loss: 0.3029 - val_accuracy: 0.8700
Epoch 6/10
140/140 [==============================] - 0s 1ms/step - loss: 0.2817 - accuracy: 0.8654 - val_loss: 0.2456 - val_accuracy: 0.8709
Epoch 7/10
140/140 [==============================] - 0s 1ms/step - loss: 0.2042 - accuracy: 0.9015 - val_loss: 0.1665 - val_accuracy: 0.9677
Epoch 8/10
140/14

In [27]:
model.save('email_classification_model.h5')

In [45]:
from tensorflow.keras.models import load_model

model = load_model('email_classification_model.h5')
# New email example
new_email = ["ddddd"]
#new_email = ["I hope this email finds you well. My name is Anisur Rahman, and I am writing from Bangladesh. I received my I-20 on MS in Artificial intelligence in August (APP-00102794).I noticed that the status in my application portal is still showing as 'initial state,' whereas I believe it should now be in the 'delivery document' stage. I am concerned that there might be an issue. I have attached a screenshot of my portal for your reference."]

# Tokenize and pad the new email
new_sequence = tokenizer.texts_to_sequences(new_email)
padded_new_sequence = pad_sequences(new_sequence, padding='post', maxlen=padded_sequences.shape[1])

# Make predictions
predictions = model.predict(padded_new_sequence)

# Convert predictions back to categories
predicted_labels = mlb.inverse_transform(predictions > 0.5)  # Use a threshold of 0.5
print(predicted_labels)


1/1 [==============================] - 0s 45ms/step
[('ham',)]


In [40]:
# evaluate model
# Example evaluation (assuming you have separate test data)
test_data = pd.read_csv(data_path)
test_sequences = tokenizer.texts_to_sequences(test_data['Message'])
padded_test_sequences = pad_sequences(test_sequences, padding='post')
y_test = mlb.transform(test_data['Message'])

model.evaluate(padded_test_sequences, y_test)


175/175 [==============================] - 0s 422us/step - loss: 2.2646 - accuracy: 0.8780


/opt/anaconda3/envs/2024/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '~', '\x91', '\x92', '\x93', '\x94', '\x96', '¡', '£', '»', 'É', 'Ü', 'è', 'é', 'ì', 'ú', 'ü', '–', '—', '‘', '’', '“', '…', '┾', '〨', '鈥'] will be ignored
  warnings.warn(


[2.264597177505493, 0.8779612183570862]